# Set up Earthengine

In [ ]:
# Import libraries.
#--install arcpy if needed
#!conda install arcpy=2.9 -c esri
import folium
import ee

In [ ]:
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
def get_upland_ws(geometry, basin_image = ee.Image("projects/ee-tacoma-watershed/assets/subbasins_out")):
    basin_id = basin_image.sample(geometry)
    id_val = (basin_id.first().get('b1'))
    list_elevations = (
        basin_id.reduceColumns(
            ee.Reducer.toList(), ['b1']).get('list')
    )
    basin = basin_image.eq(ee.Number(id_val)).selfMask()
                  
    ##for multiple basins, store an array of elevations
    basin_array_image = (ee.Image(ee
            .Array(list_elevations)
            ))
    basintesting2 = basin_array_image.eq(
            basin_image).arrayReduce(ee
            .Reducer.max(), [0])
    
    basin_mask = (basintesting2
        .arrayDotProduct(ee.Image(1)
            .toArray()))
    basin = basintesting2
    ## ee.Image("projects/ee-tacoma-watershed/assets/hand_d8").rename('hand')
    hand_img = ee.Image("projects/ee-tacoma-watershed/assets/handpit_breached").rename('hand')
    ## ee.Image("projects/ee-tacoma-watershed/assets/elev_above_pits_dem32").rename('hand')
    ##get hand value at point or geometry (for other than a point will get the lowest elevation)
    hand_elev = hand_img.sample(geometry).reduceColumns(
        reducer = ee.Reducer.min(),
        selectors = ['hand']
    ).get('min')
  
    ##add hand info
    hand_basin_image = (hand_img
        .mask(basin_mask).gte(ee
            .Number(hand_elev))
        .selfMask())
    hand_basin_vect = hand_basin_image.reduceToVectors(maxPixels= 1e12
        )
    hand_basin_vect = (hand_basin_vect
        .union(ee.ErrorMargin(1))
        .first())
    ##hand_basin_vect = hand_basin_vect.first()
    return (ee.Feature(hand_basin_vect))


##


In [ ]:
pt = ee.Geometry.Point([-122.52633893658201,47.25676548745265])

In [ ]:
shed_example = get_upland_ws(pt)
shed_img = ee.Image().paint(shed_example)

In [ ]:
# Use folium to visualize the imagery.


In [ ]:
#get facilities from tacoma 

In [ ]:
import os
import requests


In [ ]:
#read geojson
import geojson
with open('data/sw_facilities.geojson') as f:
    gj = geojson.load(f)
features = gj['features']



In [ ]:
#get first point 
pt2 =
shed_example = get_upland_ws( ee.Geometry.Point(pt2))
#json = response.read()
mapid = shed_example.getMapId()

ex_map = folium.Map(location=(pt2[1],pt2[0]), zoom_start=15,tiles='Stamen Terrain')

folium.raster_layers.TileLayer('Stamen Terrain').add_to(ex_map)

folium.GeoJson(features).add_to(ex_map)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='border',
  ).add_to(ex_map)

ex_map.add_child(folium.LayerControl())
ex_map

In [ ]:
feature_info = shed_example.getInfo()

geometry = (feature_info.get('geometry'))
i = 0 
id = features[i]
ALTID = id.properties.get('ALTID')
loc =  id.geometry.coordinates
shed = shed_example = get_upland_ws(ee.Geometry.Point(loc))

shed = shed.getInfo()
shed.get('geometry')
type(shed)
#shed.properties['ALTID'] = ALTID